### SPOT Band Dropout and Inference

In [34]:
import sys, json, os, glob

import geojson, pyproj
from functools import partial
import numpy as np
import geopandas as gpd
import pandas as pd

import h5py
from tensorflow.keras.models import Model

from shapely.affinity import affine_transform
from shapely.ops import transform
from shapely import geometry
from shapely.strtree import STRtree
from PIL import Image, ImageDraw

from skimage.measure import block_reduce
import descarteslabs as dl

import matplotlib.pyplot as plt

In [2]:
root = '/home/jovyan/solar-pv-global-inventory'

In [3]:
sys.path.append(root)

In [4]:
from solarpv.training.spot.unet_fix import UNet

In [5]:
#cv_tiles = json.load(open(os.path.join(root, 'data','tiles_airbus_fc.geojson'),'r'))['features']
cv_polys = json.load(open(os.path.join(root,'data','airbus_cv_polys_110.geojson'),'r'))['features'] + json.load(open(os.path.join(root,'data','all_test_polys.geojson'),'r'))['features'] 

### Make Annotation

In [6]:
cv_poly_geoms = [geometry.shape(ft['geometry']) for ft in cv_polys]

In [7]:
tree = STRtree(cv_poly_geoms)

In [8]:
def _get_annotation(tile, tree):

    
    WGS84 = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
    tile_srs = tile["properties"]["proj4"]
    dt = tile['properties']['geotrans']
    dt_shapely = [
                    1 / dt[1],
                    dt[2],
                    dt[4],
                    1 / dt[5],
                    -dt[0] / dt[1],
                    -dt[3] / dt[5],
                ]
    
    
    utm_proj = pyproj.Proj(tile_srs)
    wgs_proj = pyproj.Proj(WGS84)
    projection_func = partial(pyproj.transform, wgs_proj, utm_proj)
    
    tile_poly = geometry.shape(tile['geometry'])
    tile_poly_utm = transform(projection_func, tile_poly)
    
    annotations = np.zeros((tile['properties']['tilesize'], tile['properties']['tilesize'])) #np.ones((arr.shape[0], arr.shape[1]))*128
    im = Image.fromarray(annotations, mode='L')
    draw = ImageDraw.Draw(im)
    
    intersect_polys = [pp for pp in tree.query(tile_poly) if pp.intersects(tile_poly)]

    # draw annotation polygons
    for pp in intersect_polys:

        pp_utm = transform(projection_func, pp)

        pp_utm_intersection = pp_utm.intersection(tile_poly_utm)

        if pp_utm_intersection.type == 'MultiPolygon':
            sub_utm_geoms = list(pp_utm_intersection)
        else:
            sub_utm_geoms = [pp_utm_intersection]

        for sub_utm_geom in sub_utm_geoms:
            pix_geom = affine_transform(sub_utm_geom, dt_shapely)
            xs, ys = pix_geom.exterior.xy
            draw.polygon(list(zip(xs, ys)), fill=255)

            for hole in sub_utm_geom.interiors:
                xs,ys = hole.xy
                draw.polygon(list(zip(xs, ys)), fill=0)


    annotations = np.array(im)
    return annotations
    
    fig, ax = plt.subplots(1,1,figsize=(16,16))
    ax.imshow(annotations)
    plt.show()
    
    
    
    print(len(intersect_polys))

In [9]:
data_metadata= {
        'products': ['airbus:oneatlas:spot:v2'],
        'bands': ['red', 'green', 'blue', 'nir'],
        'resolution': 1.5,
        'start_datetime': '2016-01-01',
        'end_datetime': '2018-12-31',
        'tilesize': 512,
        'pad': 0,
    }

In [10]:
raster_client = dl.Raster()
metadata_client = dl.Metadata()

In [11]:
gdf = gpd.GeoDataFrame(pd.DataFrame.from_records([ft['properties'] for ft in cv_polys]), geometry = [geometry.shape(ft['geometry']) for ft in cv_polys], crs={'init':'epsg:4326'})

In [12]:
mp = gdf.unary_union

In [13]:
cv_tiles = raster_client.dltiles_from_shape(1.5, 512, 0, mp)['features']

In [14]:
def _get_arr(tile,params):
    scenes = metadata_client.search(params['products'], 
                                geom=tile, 
                                start_datetime=params['start_datetime'],  
                                end_datetime=params['end_datetime'],
                                limit=15)['features']
    arr, meta = raster_client.ndarray(
            [s.id for s in scenes],
            dltile=tile,
            bands=params['bands'],
            scales=[[0, 255]]*4,
            data_type="Byte",
        )
    return arr

In [15]:
model = UNet(n_bands=4, n_classes=2)

up 3 256
Tensor("conv2d_transpose/Identity:0", shape=(None, None, None, 512), dtype=float32)
Tensor("concatenate/Identity:0", shape=(None, None, None, 768), dtype=float32)
up 2 128
Tensor("conv2d_transpose_1/Identity:0", shape=(None, None, None, 256), dtype=float32)
Tensor("concatenate_1/Identity:0", shape=(None, None, None, 384), dtype=float32)
up 1 64
Tensor("conv2d_transpose_2/Identity:0", shape=(None, None, None, 128), dtype=float32)
Tensor("concatenate_2/Identity:0", shape=(None, None, None, 192), dtype=float32)
up 0 32
Tensor("conv2d_transpose_3/Identity:0", shape=(None, None, None, 64), dtype=float32)
Tensor("concatenate_3/Identity:0", shape=(None, None, None, 96), dtype=float32)


In [16]:
f = h5py.File(os.path.join(root,'data','solar_pv_airbus_spot_rgbn_v5_0111.hdf5'))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [17]:
for layer in model.layers:
    if len(layer.weights)>0:
        #print (layer.name, layer.weights[0].shape, layer.output_shape) # 0-> kernel, 1-> bias
        ll = [np.array(f['model_weights'][layer.name][layer.name]['kernel:0']), 
              np.array(f['model_weights'][layer.name][layer.name]['bias:0'])]
        layer.set_weights(ll)
    else:
        #print (layer.name)
        pass

In [18]:
def _iou(annotation, inference):
    # annotation and inference must be boolean arrays
    intersection = (annotation & inference).sum()
    union = (annotation | inference).sum()
    return intersection/union

In [19]:
def _perturb(arr_inp, perturbation='band_dropout', which=0, magnitude=0):
    arr = arr_inp.copy()
    if perturbation=='band_dropout':
        arr[:,:,which]=0
        return arr

    elif perturbation=='additive':
        noise = magnitude*np.random.rand(arr.shape[0]*arr.shape[1]).reshape(arr.shape[0], arr.shape[1]) - magnitude/2
        arr[:,:,which]=arr[:,:,which]+noise
        return arr

    elif perturbation=='multiplicative':
        noise = np.ones((arr.shape[0],arr.shape[1]))+ magnitude*np.random.rand(arr.shape[0]*arr.shape[1]).reshape(arr.shape[0],arr.shape[1]) - magnitude/2
        arr[:,:,which]=arr[:,:,which]*noise
        return arr

In [20]:
latent_low_model = Model(inputs=model.input,outputs=model.get_layer('conv2d_1').output)
latent_high_model = Model(inputs=model.input,outputs=model.get_layer('conv2d_9').output)

In [30]:
def run_tile(tile,params, v=False):
    
    try:
        print (f'Doing {tile["properties"]["key"]}...')
    
        # get the annotation array
        ann = _get_annotation(tile, tree)

        # get the sample
        arr = _get_arr(tile,params)

        # perturb the sample
        perturb_ll = [arr] + [_perturb(arr, 'band_dropout',which=w) for w in range(4)]
        labels = ['None'] + [f'bdo_{w}' for w in range(4)]

        for mag in [0.1, 0.2, 0.3]:
            perturb_ll += [_perturb(arr, 'additive', which=w, magnitude=mag) for w in range(4)]
            labels+= [f'additive_{mag}_{w}' for w in range(4)]

        for mag in [0.1, 0.2, 0.3]:
            perturb_ll += [_perturb(arr, 'multiplicative', which=w, magnitude=mag) for w in range(4)]
            labels+= [f'multiplicative_{mag}_{w}' for w in range(4)]

        perturb_arr = np.stack(perturb_ll)

        # run the prediction
        pred = model.predict(perturb_arr, batch_size=5)

        #calculate the IoU impairments
        record = {}
        record['P_px'] = int((ann>0).sum())
        for ii_l, label in enumerate(labels):
            record[label] = float(_iou(ann>0, pred[ii_l,:,:,0]>0.5))
        if v:
            print (record)

        # fetch the latent feature representation
        latent_space_low = latent_low_model.predict(arr[np.newaxis,...])
        latent_space_high = latent_high_model.predict(arr[np.newaxis,...])

        # maxpool the low level feature representation
        latent_space_low = block_reduce(latent_space_low, (1,16,16,1), np.max)

        if v:
            # visulise sample
            fig0, axs0 = plt.subplots(1,2,figsize=(16,8))
            axs0[0].imshow(arr[:,:,0:3])
            axs0[1].imshow(ann)

            # visualise predictions
            fig1, axs1 = plt.subplots(5,6,figsize=(30,20))
            for ii in range(pred.shape[0]):
                axs1[ii%5,ii//5].imshow(np.squeeze(pred[ii,...]))

            #visualise low latent space
            fig2, axs2= plt.subplots(4,8, figsize=(20,16))
            for ii in range(32):
                axs2[ii%4, ii//4].imshow(latent_space_low[0,:,:,ii])

            # visualise high latent space
            fig3, axs3= plt.subplots(8,8, figsize=(20,20))
            offset=0
            for ii in range(64):
                axs3[ii%8, ii//8].imshow(latent_space_high[0,:,:,ii+offset])


            plt.show()

        # save record, ann, arr, latent inference
        json.dump(record, open(os.path.join(root,'data','spot','crossval',tile['properties']['key']+'_record.json'),'w'))
        np.savez(os.path.join(root, 'data','spot','crossval',tile['properties']['key']+'_data.npz'), arr=arr, ann=ann, lsl=latent_space_low,lsh=latent_space_high)
    except Exception as e:
        print ('Error!')
        print (e)
        
        
    

In [ ]:
for tile in cv_tiles:
    run_tile(tile,data_metadata)#,v=True)

Doing 512:0:1.5:04:-79:3153...
Doing 512:0:1.5:10:-49:6497...
Doing 512:0:1.5:10:-49:6498...
Doing 512:0:1.5:10:98:5510...
Doing 512:0:1.5:10:99:5508...
Doing 512:0:1.5:10:99:5509...
Doing 512:0:1.5:10:99:5510...
Doing 512:0:1.5:10:99:5511...
Doing 512:0:1.5:10:99:5512...
Doing 512:0:1.5:10:99:5513...
Doing 512:0:1.5:10:100:5510...
Doing 512:0:1.5:10:100:5511...
Doing 512:0:1.5:10:100:5512...
Doing 512:0:1.5:10:101:5510...
Doing 512:0:1.5:10:101:5511...
Doing 512:0:1.5:10:101:5512...
Doing 512:0:1.5:10:103:5509...
Doing 512:0:1.5:10:103:5510...
Doing 512:0:1.5:10:104:5509...
Doing 512:0:1.5:10:104:5510...
Doing 512:0:1.5:10:105:5510...
Doing 512:0:1.5:10:105:5511...
Doing 512:0:1.5:10:106:5510...
Doing 512:0:1.5:10:106:5511...
Doing 512:0:1.5:10:106:5512...
Doing 512:0:1.5:10:178:5526...
Doing 512:0:1.5:10:180:5527...
Doing 512:0:1.5:10:180:5528...
Doing 512:0:1.5:10:180:5530...
Doing 512:0:1.5:10:180:5531...
Doing 512:0:1.5:10:181:5527...
Doing 512:0:1.5:10:181:5528...
Doing 512:0:1.5

Doing 512:0:1.5:17:343:5074...
Doing 512:0:1.5:17:344:5074...
Doing 512:0:1.5:17:344:5078...
Doing 512:0:1.5:17:344:5079...
Doing 512:0:1.5:17:345:5076...
Doing 512:0:1.5:17:345:5077...
Doing 512:0:1.5:17:345:5078...
Doing 512:0:1.5:17:345:5079...
Doing 512:0:1.5:18:-294:5097...


### Compile records for fig

In [35]:
records = glob.glob(os.path.join(root,'data','spot','crossval','*_record.json'))

In [38]:
load_records = [json.load(open(r,'r')) for r in records]

In [41]:
df = pd.DataFrame.from_records(load_records)

In [45]:
df['tile'] = [r.split('/')[-1].split('_')[0] for r in records]

In [47]:
df

,P_px,None,bdo_0,bdo_1,bdo_2,bdo_3,additive_0.1_0,additive_0.1_1,additive_0.1_2,additive_0.1_3,...,multiplicative_0.1_3,multiplicative_0.2_0,multiplicative_0.2_1,multiplicative_0.2_2,multiplicative_0.2_3,multiplicative_0.3_0,multiplicative_0.3_1,multiplicative_0.3_2,multiplicative_0.3_3,tile
0,237,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,512:0:1.5:18:60:5842
1,54146,0.753549,0.050268,0.0,0.0,0.766301,0.754032,0.754487,0.753843,0.753004,...,0.756307,0.656239,0.777797,0.776135,0.768322,0.519971,0.766116,0.791823,0.781573,512:0:1.5:11:-180:5018
2,2886,0.269046,0.000000,0.0,0.0,0.000000,0.301919,0.259374,0.244070,0.275318,...,0.248195,0.336286,0.275402,0.239476,0.255464,0.374957,0.359574,0.253025,0.258657,512:0:1.5:53:-46:5029
3,88582,0.803565,0.000113,0.0,0.0,0.592490,0.802303,0.809204,0.800279,0.803264,...,0.807064,0.805952,0.820112,0.800947,0.806526,0.788536,0.806418,0.805872,0.814159,512:0:1.5:43:-236:3849
4,29185,0.806368,0.034949,0.0,0.0,0.849237,0.800076,0.825488,0.796172,0.802842,...,0.801556,0.841727,0.826244,0.810833,0.813203,0.843430,0.846183,0.821550,0.833848,512:0:1.5:43:-352:3958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,8005,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,512:0:1.5:42:347:3375
1380,35719,0.167559,0.001175,0.0,0.0,0.159168,0.166927,0.168550,0.166050,0.167788,...,0.168559,0.170589,0.172950,0.167587,0.168115,0.173909,0.173282,0.171675,0.169643,512:0:1.5:42:231:3366
1381,449,0.198934,0.000000,0.0,0.0,0.116644,0.195918,0.194828,0.205665,0.199403,...,0.192073,0.128258,0.059932,0.273699,0.226391,0.111276,0.075033,0.258806,0.177934,512:0:1.5:10:101:5510
1382,3460,0.236453,0.000000,0.0,0.0,0.109656,0.225931,0.228183,0.256310,0.241704,...,0.237055,0.211557,0.223091,0.267853,0.239260,0.206276,0.234433,0.229043,0.239618,512:0:1.5:54:-314:5056


In [48]:
df.to_csv('band_perturbation_SPOT.csv')

### Archive

In [ ]:
ann = _get_annotation(cv_tiles[25], tree)

In [ ]:
plt.imshow(ann)

In [ ]:
arr = _get_arr(cv_tiles[25],data_metadata)
            

In [ ]:
plt.imshow(arr[:,:,0:3])

In [ ]:
arr.shape, arr.min(), arr.max()

In [ ]:
perturb_ll = [_perturb(arr, 'band_dropout',which=w) for w in range(4)]

In [ ]:
for mag in [0.1, 0.2, 0.3]:
    perturb_ll += [_perturb(arr, 'additive', which=w, magnitude=mag) for w in range(4)]

In [ ]:
for mag in [0.1, 0.2, 0.3]:
    perturb_ll += [_perturb(arr, 'multiplicative', which=w, magnitude=mag) for w in range(4)]

In [ ]:
perturb_arr = np.stack([arr]+perturb_ll)

In [ ]:
perturb_arr.shape

In [ ]:
pred = model.predict(perturb_arr, batch_size=5)

In [ ]:
pred.shape

In [ ]:
a = np.array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
              [1,0,1,0]],
       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23],
       [2,0,3,0]],
       [[24, 25, 26, 27],
        [28, 29, 30, 31],
        [32, 33, 34, 35],
       [1,0,2,3]]])

In [ ]:
a.shape

In [ ]:
block_reduce(a, block_size=(2,2,1), func=np.sum).shape

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
a = load_model(os.path.join(root,'data','solar_pv_airbus_spot_rgbn_v5_0111.hdf5'))

In [ ]:
pred = a.predict(arr[np.newaxis,...].astype('float16'), batch_size=1, verbose=1)

In [ ]:
plt.imshow(np.squeeze(pred>0.5))

In [ ]:
for layer in a.layers:
    if len(layer.weights)>0:
        print (layer.name, layer.weights[0].shape, layer.output_shape)
    else:
        print (layer.name)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    if len(layer.weights)>0:
        print (layer.name, layer.weights[0].shape, layer.output_shape) # 0-> kernel, 1-> bias
    else:
        print (layer.name)

In [ ]:
model.load_weights(os.path.join(root,'data','solar_pv_airbus_spot_rgbn_v5_0111.hdf5'))